# Inport needed package

- import os, sys # to add the parent directory to the path

In [42]:
import os
import sys
import time

- Using torchvision to create a dataset

In [43]:
# Using torchvision to create a dataset
import cv2
from torchvision import transforms
import torch
from torch.utils.data import random_split, DataLoader
import torchvision

import pandas as pd

- import self library

In [44]:
from train.trainer import ClassifierTrainer as Trainer
import dataset as ds  # type: ignore

# Define classification train process

1. Define place where the model is saved

In [49]:
time_str = time.strftime("%Y%m%d_%H%M%S")


2. Define train function

In [ ]:
def doTheTrain(dataset, model):
  # define batch_size
  batch_size = 64

  # init train val test ds
  train_val_size = int(0.8 * len(dataset))
  test_size = len(dataset) - train_val_size
  train_ds, test_ds = random_split(dataset, [train_val_size, test_size])

  # define optimizer using Adam and loss function
  optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
  loss_fn = torch.nn.CrossEntropyLoss()

  trainer = Trainer(model, optimizer, loss_fn, random_seed_value=86)
  print('device: ', trainer.device)
  avg_loss, metric = trainer.cross_validate(train_ds, k=10, epochs=10, batch_size=batch_size)
  print('avg_loss: ', avg_loss)

  # score model
  test_dataloader = DataLoader(test_ds, batch_size=batch_size, shuffle=True)
  model_scored = trainer.score(test_dataloader)
  print(f'model_scored: {model_scored:.4f}, avg_accuracy: {100*(1 - model_scored):.4f}')

  # return model scored, train_avg_lost
  return model_scored, avg_loss

3. execute progress

- define the model

In [ ]:
models = [
    torchvision.models.resnet50(weights=torchvision.models.ResNet50_Weights.DEFAULT),
    torchvision.models.densenet121(weights=torchvision.models.DenseNet121_Weights.DEFAULT),
    torchvision.models.vgg16(weights=torchvision.models.VGG16_Weights.DEFAULT),
]

- Define tested datasets

In [ ]:
datasets = {
    'gi4e_full': ds.Gi4eDataset(
        './datasets/gi4e',
        transform=transforms.Compose([transforms.Resize((224, 224)), transforms.ToTensor()]),
        is_classification=True),
    'gi4e_raw_eyes': ds.ImageDataset(
        './datasets/gi4e_raw_eyes',
        transform=transforms.Compose([transforms.Resize((224, 224)), transforms.ToTensor()]),
        file_extension='png'),
    'gi4e_detected_eyes': ds.ImageDataset(
        './datasets/gi4e_eyes/20250521_200316',
        transform=transforms.Compose([transforms.Resize((224, 224)), transforms.ToTensor()]),
        file_extension='png'),
}

- Train all defined model on each registered datasets

In [ ]:
result_df = pd.DataFrame(columns=['model', 'dataset', 'avg_loss', 'avg_accuracy'])

for name, dataset in datasets.items():
	for model in models:
		print(f'Running {name} dataset with {model.__class__.__name__}')
		# do the train
		scored, loss = doTheTrain(dataset, model)
		print(f'Finished {name} dataset with {model.__class__.__name__}')
		print('----------------------')

		# save the result
		result_df = pd.concat([result_df, pd.DataFrame({
			'model': [model.__class__.__name__],
			'dataset': [name],
			'avg_loss': [loss],
			'avg_accuracy': [scored]
		})], ignore_index=True)

print('Finished all datasets')

# print the result
print(result_df)



Running gi4e_raw_eyes dataset with ResNet
device:  cuda
Fold 1/10:
Epoch 1/10, Train Loss: 2.4886, Test Loss: 2.8957, Total Time: 00 hours, 00 minutes, 15 seconds
Epoch 2/10, Train Loss: 0.1088, Test Loss: 0.1940, Total Time: 00 hours, 00 minutes, 12 seconds
Epoch 3/10, Train Loss: 0.0273, Test Loss: 0.0624, Total Time: 00 hours, 00 minutes, 12 seconds
Epoch 4/10, Train Loss: 0.0362, Test Loss: 0.0239, Total Time: 00 hours, 00 minutes, 12 seconds
Epoch 5/10, Train Loss: 0.0305, Test Loss: 0.1215, Total Time: 00 hours, 00 minutes, 12 seconds
Epoch 6/10, Train Loss: 0.0146, Test Loss: 0.0100, Total Time: 00 hours, 00 minutes, 12 seconds
Epoch 7/10, Train Loss: 0.0168, Test Loss: 0.0213, Total Time: 00 hours, 00 minutes, 12 seconds
Epoch 8/10, Train Loss: 0.0328, Test Loss: 0.1616, Total Time: 00 hours, 00 minutes, 12 seconds
Epoch 9/10, Train Loss: 0.0853, Test Loss: 0.2304, Total Time: 00 hours, 00 minutes, 12 seconds
Epoch 10/10, Train Loss: 0.1102, Test Loss: 0.5742, Total Time: 00 ho

c:\Users\nnvuf\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\nnvuf\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\nnvuf\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, 

Fold 1/10, Total Test Loss: 0.5742, Fold accuracy: 42.5798
Fold 2/10:
Epoch 1/10, Train Loss: 0.0788, Test Loss: 0.4703, Total Time: 00 hours, 00 minutes, 13 seconds
Epoch 2/10, Train Loss: 0.0642, Test Loss: 0.6441, Total Time: 00 hours, 00 minutes, 12 seconds
Epoch 3/10, Train Loss: 0.0236, Test Loss: 0.2766, Total Time: 00 hours, 00 minutes, 12 seconds
Epoch 4/10, Train Loss: 0.0263, Test Loss: 0.1328, Total Time: 00 hours, 00 minutes, 12 seconds
Epoch 5/10, Train Loss: 0.0125, Test Loss: 0.0486, Total Time: 00 hours, 00 minutes, 13 seconds
Epoch 6/10, Train Loss: 0.0066, Test Loss: 0.0012, Total Time: 00 hours, 00 minutes, 15 seconds
Epoch 7/10, Train Loss: 0.0005, Test Loss: 0.0009, Total Time: 00 hours, 00 minutes, 12 seconds
Epoch 8/10, Train Loss: 0.0002, Test Loss: 0.0008, Total Time: 00 hours, 00 minutes, 12 seconds
Epoch 9/10, Train Loss: 0.0002, Test Loss: 0.0005, Total Time: 00 hours, 00 minutes, 12 seconds
Epoch 10/10, Train Loss: 0.0002, Test Loss: 0.0009, Total Time: 00

C:\Users\nnvuf\AppData\Local\Temp\ipykernel_26232\655168368.py:14: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  result_df = pd.concat([result_df, pd.DataFrame({


Epoch 1/10, Train Loss: 2.2658, Test Loss: 2.8522, Total Time: 00 hours, 00 minutes, 21 seconds
Epoch 2/10, Train Loss: 0.0641, Test Loss: 0.0198, Total Time: 00 hours, 00 minutes, 13 seconds
Epoch 3/10, Train Loss: 0.0086, Test Loss: 0.0052, Total Time: 00 hours, 00 minutes, 13 seconds
Epoch 4/10, Train Loss: 0.0029, Test Loss: 0.0020, Total Time: 00 hours, 00 minutes, 13 seconds
Epoch 5/10, Train Loss: 0.0015, Test Loss: 0.0013, Total Time: 00 hours, 00 minutes, 13 seconds
Epoch 6/10, Train Loss: 0.0010, Test Loss: 0.0011, Total Time: 00 hours, 00 minutes, 13 seconds
Epoch 7/10, Train Loss: 0.0008, Test Loss: 0.0009, Total Time: 00 hours, 00 minutes, 13 seconds
Epoch 8/10, Train Loss: 0.0007, Test Loss: 0.0008, Total Time: 00 hours, 00 minutes, 13 seconds
Epoch 9/10, Train Loss: 0.0005, Test Loss: 0.0006, Total Time: 00 hours, 00 minutes, 13 seconds
Epoch 10/10, Train Loss: 0.0005, Test Loss: 0.0006, Total Time: 00 hours, 00 minutes, 12 seconds
Fold 1/10, Total Test Loss: 0.0006, Fol

...

Fold 6/10, Total Test Loss: 4.2397, Fold accuracy: 95.9805
Fold 7/10:
Epoch 1/10, Train Loss: 0.0384, Test Loss: 0.0070, Total Time: 00 hours, 00 minutes, 19 seconds
Epoch 2/10, Train Loss: 0.0756, Test Loss: 0.0037, Total Time: 00 hours, 00 minutes, 19 seconds
Epoch 3/10, Train Loss: 0.0582, Test Loss: 0.0028, Total Time: 00 hours, 00 minutes, 19 seconds
Epoch 4/10, Train Loss: 0.0338, Test Loss: 0.0151, Total Time: 00 hours, 00 minutes, 19 seconds
Epoch 5/10, Train Loss: 0.0480, Test Loss: 0.0077, Total Time: 00 hours, 00 minutes, 18 seconds
Epoch 6/10, Train Loss: 0.0424, Test Loss: 0.0351, Total Time: 00 hours, 00 minutes, 18 seconds
Epoch 7/10, Train Loss: 0.0132, Test Loss: 0.0067, Total Time: 00 hours, 00 minutes, 19 seconds
Epoch 8/10, Train Loss: 0.0032, Test Loss: 0.0004, Total Time: 00 hours, 00 minutes, 19 seconds
Epoch 9/10, Train Loss: 0.0508, Test Loss: 0.0007, Total Time: 00 hours, 00 minutes, 19 seconds
Epoch 10/10, Train Loss: 0.0182, Test Loss: 0.0028, Total Time: 00

c:\Users\nnvuf\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\nnvuf\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\nnvuf\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo

Fold 1/10, Total Test Loss: 0.0396, Fold accuracy: 96.0425
Fold 2/10:
Epoch 1/10, Train Loss: 0.0119, Test Loss: 0.0093, Total Time: 00 hours, 00 minutes, 12 seconds
Epoch 2/10, Train Loss: 0.0062, Test Loss: 0.0008, Total Time: 00 hours, 00 minutes, 11 seconds
Epoch 3/10, Train Loss: 0.0012, Test Loss: 0.0001, Total Time: 00 hours, 00 minutes, 12 seconds
Epoch 4/10, Train Loss: 0.0002, Test Loss: 0.0001, Total Time: 00 hours, 00 minutes, 12 seconds
Epoch 5/10, Train Loss: 0.0001, Test Loss: 0.0000, Total Time: 00 hours, 00 minutes, 12 seconds
Epoch 6/10, Train Loss: 0.0001, Test Loss: 0.0000, Total Time: 00 hours, 00 minutes, 11 seconds
Epoch 7/10, Train Loss: 0.0001, Test Loss: 0.0000, Total Time: 00 hours, 00 minutes, 11 seconds
Epoch 8/10, Train Loss: 0.0001, Test Loss: 0.0000, Total Time: 00 hours, 00 minutes, 11 seconds
Epoch 9/10, Train Loss: 0.0000, Test Loss: 0.0000, Total Time: 00 hours, 00 minutes, 11 seconds
Epoch 10/10, Train Loss: 0.0000, Test Loss: 0.0000, Total Time: 00

c:\Users\nnvuf\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\nnvuf\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\nnvuf\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo

Fold 1/10, Total Test Loss: 0.0235, Fold accuracy: 97.6472
Fold 2/10:
Epoch 1/10, Train Loss: 0.0277, Test Loss: 2.0885, Total Time: 00 hours, 00 minutes, 13 seconds
Epoch 2/10, Train Loss: 0.3024, Test Loss: 2.4556, Total Time: 00 hours, 00 minutes, 13 seconds
Epoch 3/10, Train Loss: 0.1372, Test Loss: 0.7813, Total Time: 00 hours, 00 minutes, 13 seconds
Epoch 4/10, Train Loss: 0.0181, Test Loss: 0.0691, Total Time: 00 hours, 00 minutes, 13 seconds
Epoch 5/10, Train Loss: 0.0112, Test Loss: 0.0066, Total Time: 00 hours, 00 minutes, 13 seconds
Epoch 6/10, Train Loss: 0.0013, Test Loss: 0.0014, Total Time: 00 hours, 00 minutes, 13 seconds
Epoch 7/10, Train Loss: 0.0005, Test Loss: 0.0009, Total Time: 00 hours, 00 minutes, 13 seconds
Epoch 8/10, Train Loss: 0.0004, Test Loss: 0.0007, Total Time: 00 hours, 00 minutes, 13 seconds
Epoch 9/10, Train Loss: 0.0003, Test Loss: 0.0006, Total Time: 00 hours, 00 minutes, 13 seconds
Epoch 10/10, Train Loss: 0.0003, Test Loss: 0.0006, Total Time: 00

4. print the result

In [51]:
# swap the first two columns
result_df = result_df[['dataset', 'model', 'avg_loss', 'avg_accuracy']]
# scale the avg_accuracy to 0-100
result_df['avg_accuracy'] = 100 * (1 - result_df['avg_accuracy'])

# save the result to csv
result_df.to_csv(f'results_{time_str}.csv', index=False)
# print the result
print(result_df)

              dataset     model  avg_loss  avg_accuracy
0       gi4e_raw_eyes    ResNet  0.057511     99.964015
1       gi4e_raw_eyes  DenseNet  0.000077     99.059641
2       gi4e_raw_eyes       VGG  0.427026     85.286917
3  gi4e_detected_eyes    ResNet  0.003961     99.308679
4  gi4e_detected_eyes  DenseNet  0.002429     97.700996
5  gi4e_detected_eyes       VGG  0.002681     94.411226
